# PRÁCTICA 2 Parte II y III

Javier García Serrano, Ana Gil Molina

## Librerías necesarias

In [1]:
import os
import json

import torch
import numpy as np
import csv
import urllib.request
import re

from scipy.special import softmax
from rouge_score import rouge_scorer

from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import pipeline
from transformers import MT5Tokenizer, MT5ForConditionalGeneration
from transformers import TrainingArguments, Trainer
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BartTokenizer, BartForConditionalGeneration

from datasets import load_dataset

In [2]:
# Desactivamos esta herramienta de monitorización que viene configurada por defecto en transformers
os.environ['WANDB_DISABLED'] = 'True'

In [ ]:
# Ahora vamos a ver si nuestro entorno tiene GPU o no
device = torch.device ("cuda:0" if torch.cuda.is_available () else "cpu")
print(device)

## <div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>3) Análisis de subjetividad de los comentarios (calificación 1)</strong></div>

Para comenzar, cargamos los datos que hemos compilado en el apartado 1 de esta práctica, y que hemos almacenado en la carpeta "Threads".

In [ ]:
# Ruta de la carpeta base 'Threads'
script_dir = os.getcwd()                                  # Ruta actual del notebook
threads_dir = os.path.join(script_dir, 'Threads')         # Ruta a la carpeta 'Threads'

# Listar todas las carpetas (subreddits) dentro de la carpeta 'Threads'
subreddit_folders = [f for f in os.listdir(threads_dir) if os.path.isdir(os.path.join(threads_dir, f))]

print("Subreddits encontrados:", subreddit_folders)

# Crear un diccionario para almacenar los archivos por subreddit
subreddit_files = {}

# Listar todos los archivos JSON dentro de cada carpeta de subreddit
for subreddit_folder in subreddit_folders:
    subreddit_folder_path = os.path.join(threads_dir, subreddit_folder)

    # Obtener la lista de archivos JSON dentro de la carpeta del subreddit
    json_files = [os.path.join(subreddit_folder_path, f) for f in os.listdir(subreddit_folder_path) if f.endswith('.json')]

    # Almacenar la lista de archivos JSON en el diccionario con el subreddit como clave
    subreddit_files[subreddit_folder] = json_files

# Mostrar el diccionario de archivos por subreddit
print("Archivos organizados por subreddit:", subreddit_files)

In [ ]:
# Lista para almacenar los datos cargados
threads_data = {}

# Leer cada archivo JSON de cada subreddit
for subreddit, json_files in subreddit_files.items():
    subreddit_data = []
    for json_file in json_files:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            subreddit_data.append(data)

    threads_data[subreddit] = subreddit_data

# Mostrar un ejemplo de hilo
print("Ejemplo de un hilo:", json.dumps(threads_data['AskHistorians'][0], indent=4, ensure_ascii=False))

En este hilo de ejemplo, el equipo de moderación de Ask Historians se compromete a ser transparente con la comunidad sobre sus decisiones, destacando políticas de tolerancia cero hacia comportamientos como el racismo, sexismo, acoso, y la negación de genocidios. Es decir, es un texto informativo que explica las políticas de moderación del equipo de Ask Historians, su enfoque sobre la transparencia y las decisiones tomadas en relación con comportamientos como la negación de genocidios y otros discursos dañinos en la comunidad.

### Modelo de análisis de sentimiento

En este apartado se pide emplear LLMs pre-entrenados para tareas de clasificación en las que obtener la polaridad subjetiva de cada comentario, ya sea en forma de sentimiento o emoción. Para ello, en primer lugar se ha seleccionado el modelo **Twitter-roBERTa-base for Sentiment Analysis**, disponible en https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment, para extraer el sentimiento (positivo, negativo o neutro) de cada uno de los hilos y de sus respectivos comentarios.

Este modelo es una variante del modelo roBERTa, que ha sido entrenado con un conjunto de datos de aproximadamente 58 millones de tweets, lo que le permite capturar de manera efectiva las peculiaridades del lenguaje usado en las redes sociales. Se usó fine-tuning para ajustarlo a tareas de análisis de sentimiento utilizando el benchmark TweetEval. Además, este modelo es adecuado para textos en inglés, como los que estamos considerando en nuestros datos.

A continuación, se define la tarea que va a realizar el modelo, que en este caso va a ser realizar un análisis de sentimiento, y se carga el tokenizador, responsable de dividir el texto en unidades que el modelo puede procesar.

In [7]:
# Definir la tarea (análisis de sentimiento)
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

# Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODEL)

Ahora, se descargan las etiquetas de clasificación, en este caso, "positive", "neutral" y "negative".

In [ ]:
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

labels

Por último, se carga el modelo pre-entrenado y se guarda para su uso posterior.

In [9]:
# Cargar el modelo
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# Guardar el modelo
model.save_pretrained(MODEL)

Veamos un ejemplo de uso de este modelo para el primer hilo de nuestra colección. Observamos que al tokenizar el texto, este se trunca a 512 tokens, dado que este es el tamaño máximo de secuencia que puede procesar el modelo roBERTa. Además, al aplicar el modelo a los datos, este devuelve un `output` que contiene los puntajes para cada uno de los sentimientos. Estos puntajes se pasan por la función softmax para convertirlos en probabilidades entre $0$ y $1$, de manera que sumen $1$, facilitando la interpretación de los resultados.

In [ ]:
# Texto de ejemplo
text = threads_data['AskHistorians'][0]['description']

# Se tokeniza el texto
encoded_input = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)

# Se pasa el texto tokenizado por el modelo
output = model(**encoded_input)

# Puntajes de los sentimientos
scores = output[0][0].detach().numpy()

# Aplicar la función softmax a los puntajes
scores = softmax(scores)
scores

A continuación, se muestran los resultados obtenidos, con los puntajes ordenados de mayor a menor, de forma que el sentimiento con la probabilidad más alta esté primero.

In [ ]:
# Ordenar los puntajes
ranking = np.argsort(scores)
ranking = ranking[::-1]

# Mostrar los resultados
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

De esta forma, vemos que el texto de ejemplo que hemos considerado tiene un puntaje de $0.4969$ para el sentimiento neutral, seguido de un puntaje de $0.4424$ para el sentimiento negativo, y finalmente un puntaje de $0.0607$ para el sentimiento positivo. El puntaje más alto corresponde al sentimiento neutro, lo que indica que el modelo considera que el texto tiene una inclinación principalmente neutra. Esto tiene sentido, teniendo en cuenta que este texto se corresponde con un mensaje informativo acerca de las políticas de moderación del equipo de Ask Historians.

Habiendo visto el funcionamiento del modelo para un ejemplo particular, ahora vamos a aplicarlo a cada uno de los hilos considerados y sus correspondientes comentarios. El sentimiento obtenido para cada uno se almacenará de nuevo en los ficheros JSON en el campo `sentiment` de cada comentario. Para comenzar, definimos una función que se encargue de obtener el sentimiento para un texto dado.

In [12]:
# Función para obtener el sentimiento
def classify_sentiment(text):

    # Tokenizar el texto
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Pasar el texto tokenizado por el modelo
    output = model(**encoded_input)

    # Puntajes de los sentimientos
    scores = output[0][0].detach().numpy()

    # Aplicar la función softmax a los puntajes
    scores = softmax(scores)

    # Obtener el sentimiento con el puntaje más alto
    sentiment = np.argmax(scores)
    sentiment_label = ['negative', 'neutral', 'positive'][sentiment]

    # Devolver los resultados como un diccionario
    return {'label': sentiment_label, 'score': float(scores[sentiment])}

Ahora, se recorre cada hilo en `threads_data` y se procesa tanto la descripción del hilo como sus comentarios, agregando el sentimiento mediante la clave `sentiment`, guardando únicamente la etiqueta y el puntaje del sentimiento con el puntaje más alto.

In [ ]:
# Procesar los hilos con descripciones
for subreddit, threads in threads_data.items():
    print(f"Procesando {len(threads)} hilos del subreddit {subreddit}...")

    for thread in threads:
        # Procesar la descripción del hilo
        thread['sentiment'] = classify_sentiment(thread['description'])

        # Procesar los comentarios del hilo
        for comment in thread['comments']:
            comment['sentiment'] = classify_sentiment(comment['body'])

print("Procesamiento de sentimientos completado.")

Para finalizar, guardamos los resultados obtenidos en ficheros JSON.

In [ ]:
# Función para limpiar nombres de archivos
def limpiar_nombre_archivo(nombre):
    return re.sub(r'[<>:"/\\|?*]', '_', nombre)[:50]  # Reemplaza caracteres no válidos y limita longitud

In [ ]:
# Obtener el directorio de trabajo actual
script_dir = os.getcwd()

# Ruta de la carpeta 'Modified Threads' dentro del directorio del script
modified_threads_dir = os.path.join(script_dir, 'Modified Threads')

# Crear la carpeta 'Modified Threads' si no existe
os.makedirs(modified_threads_dir, exist_ok=True)

# Iterar sobre los subreddits y sus hilos
for subreddit, threads in threads_data.items():
    # Crear una subcarpeta para cada subreddit
    subreddit_dir = os.path.join(modified_threads_dir, subreddit)
    os.makedirs(subreddit_dir, exist_ok=True)

    # Guardar los hilos modificados en la subcarpeta correspondiente
    for idx, thread in enumerate(threads, start=1):
        safe_name = limpiar_nombre_archivo(thread['title'])  # Limpiar el título para usarlo como nombre de archivo
        file_path = os.path.join(subreddit_dir, f'{safe_name}_modified_thread_{idx}.json')

        # Guardar el hilo modificado
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(thread, f, indent=4, ensure_ascii=False)

    print(f"Archivos del subreddit '{subreddit}' guardados en: {subreddit_dir}")

print(f"Todos los archivos han sido guardados en: {modified_threads_dir}")

### Modelo de análisis de emoción

Para continuar, ahora vamos a emplear un modelo pre-entrenado para el análisis de emociones en los comentarios. Para ello, utilizaremos el modelo **Fine-tuned DistilRoBERTa-base for Emotion Classification**, disponible en https://huggingface.co/michellejieli/emotion_text_classifier, que ha sido afinado específicamente para clasificar emociones en diálogos de series y películas. Este modelo se basa en DistilRoBERTa, y está diseñado para predecir seis emociones definidas por Ekman: ira, disgusto, miedo, alegría, tristeza, sorpresa, así como una clase neutral. El modelo ha sido entrenado con transcripciones de la serie *Friends*, lo que lo hace adecuado para reconocer emociones en el lenguaje coloquial y natural.

Para comenzar, veamos un ejemplo de uso de este modelo con el primer hilo de nuestra colección. De nuevo, durante la tokenización se trunca el texto a 512 tokens, pues este es el límite de tokens permitido por el modelo.

In [ ]:
# Inicializamos el clasificador
classifier = pipeline("text-classification", model="michellejieli/emotion_text_classifier")

# Función para clasificar texto largo
def classify_emotion(text):
    # Truncamos el texto a 512 tokens durante la tokenización
    truncated_text = classifier.tokenizer.decode(
        classifier.tokenizer.encode(text, truncation=True, padding=True, max_length=512),
        skip_special_tokens=True
    )

    # Clasificamos el texto truncado pasando al pipeline
    result = classifier(truncated_text)

    return result[0]

# Aplicar la función al primer hilo
text = threads_data['AskHistorians'][0]['description']
result = classify_emotion(text)

# Imprimir el resultado
print(result)

Para este ejemplo, correspondiente a un mensaje informativo acerca de las políticas de moderación del equipo de Ask Historians, la emoción obtenida ha sido neutral. Esto efectivamente tiene sentido debido a la temática del ejemplo considerado.

De forma análoga a como hemos hecho antes, podemos ahora recorrer cada hilo en `threads_data` y procesar su descripción y sus comentarios, agregando la emoción obtenida mediante la clave `emotion`.

In [ ]:
# Procesar los hilos de cada subreddit
for subreddit, threads in threads_data.items():  # Iterar sobre subreddits y sus hilos
    print(f"Procesando {len(threads)} hilos del subreddit {subreddit}...")

    for thread in threads:
        # Procesar la descripción del hilo
        thread['emotion'] = classify_emotion(thread['description'])

        # Procesar los comentarios del hilo
        for comment in thread['comments']:
            comment['emotion'] = classify_emotion(comment['body'])

print("Procesamiento de emociones completado.")

Finalmente, modificamos los ficheros JSON ya almacenados anteriormente para añadir los nuevos resultados de emoción.

In [ ]:
# Obtener el directorio de trabajo actual
script_dir = os.getcwd()

# Ruta de la carpeta 'Modified Threads' dentro del directorio del script
modified_threads_dir = os.path.join(script_dir, 'Modified Threads')

# Crear la carpeta 'Modified Threads' si no existe
os.makedirs(modified_threads_dir, exist_ok=True)

# Iterar sobre los subreddits y sus hilos
for subreddit, threads in threads_data.items():
    # Crear una subcarpeta para cada subreddit
    subreddit_dir = os.path.join(modified_threads_dir, subreddit)
    os.makedirs(subreddit_dir, exist_ok=True)

    # Guardar los hilos modificados en la subcarpeta correspondiente
    for idx, thread in enumerate(threads, start=1):
        safe_name = limpiar_nombre_archivo(thread['title'])  # Limpiar el título para usarlo como nombre de archivo
        file_path = os.path.join(subreddit_dir, f'{safe_name}_modified_thread_{idx}.json')

        # Guardar el hilo modificado
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(thread, f, indent=4, ensure_ascii=False)

    print(f"Archivos del subreddit '{subreddit}' guardados en: {subreddit_dir}")

print(f"Todos los archivos han sido guardados en: {modified_threads_dir}")

## <div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>4) Resumen automático abstractivo (calificación 1)</strong></div>

El resumen automático abstractivo es una técnica de procesamiento del lenguaje natural que genera nuevas frases para expresar las ideas principales de un texto original, reflejando el contenido y el sentido original del texto.

Antes de comenzar, vamos a cargar los ficheros JSON que en el apartado anterior hemos guardado en la carpeta "Modified Threads", para luego añadir los resúmenes obtenidos en dichos ficheros.

In [ ]:
# Ruta de la carpeta base 'Modified Threads'
script_dir = os.path.dirname(os.path.abspath('__file__'))  # Ruta actual del notebook
modified_threads_dir = os.path.join(script_dir, 'Modified Threads')         # Ruta a la carpeta 'Modified Threads'

# Listar todas las carpetas (subreddits) dentro de la carpeta 'Modified Threads'
subreddit_folders = [f for f in os.listdir(modified_threads_dir) if os.path.isdir(os.path.join(modified_threads_dir, f))]

print("Subreddits encontrados:", subreddit_folders)

# Crear un diccionario para almacenar los archivos por subreddit
subreddit_files = {}

# Listar todos los archivos JSON dentro de cada carpeta de subreddit
for subreddit_folder in subreddit_folders:
    subreddit_folder_path = os.path.join(modified_threads_dir, subreddit_folder)

    # Obtener la lista de archivos JSON dentro de la carpeta del subreddit
    json_files = [os.path.join(subreddit_folder_path, f) for f in os.listdir(subreddit_folder_path) if f.endswith('.json')]

    # Almacenar la lista de archivos JSON en el diccionario con el subreddit como clave
    subreddit_files[subreddit_folder] = json_files

# Mostrar el diccionario de archivos por subreddit
print("Archivos organizados por subreddit:", subreddit_files)

In [14]:
# Lista para almacenar los datos cargados
threads_data = {}

# Leer cada archivo JSON de cada subreddit
for subreddit, json_files in subreddit_files.items():
    subreddit_data = []
    for json_file in json_files:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            subreddit_data.append(data)

    threads_data[subreddit] = subreddit_data

### 4.1) Usando un modelo ya entrenado

Para este primer apartado, utilizaremos el modelo **mT5-multilingual-XLSum**, una variante del modelo mT5, fine-tuneado específicamente en el conjunto de datos XL-Sum. Este conjunto abarca artículos de noticias en 45 idiomas, entre ellos, el inglés, que es el que usaremos para nuestro conjunto de datos.

El objetivo de este apartado es aplicar el modelo mT5-multilingual-XLSum para generar un resumen abstractivo de la descripción de cada uno de los hilos de nuestro dataset, para luego guardar dichos resúmenes en archivos JSON.

Para comenzar, se crea una función encargada de normalizar los textos eliminando saltos de línea y espacios en blanco repetidos, de forma que el modelo los procese mejor.

In [ ]:
# Función de preprocesamiento
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

A continuación, se carga el modelo pre-entrenado y el correspondiente tokenizador.

In [ ]:
# Nombre del modelo pre-entrenado
model_name = "csebuetnlp/mT5_multilingual_XLSum"

# Cargar el tokenizador
tokenizer = MT5Tokenizer.from_pretrained(model_name, use_fast=False)

# Cargar el modelo pre-entrenado
model = MT5ForConditionalGeneration.from_pretrained(model_name)

Para ver el funcionamiento del modelo, en primer lugar vamos a probarlo con un texto de ejemplo, en este caso, tomamos la descripción del primer hilo de nuestra colección. En primer lugar, se debe tokenizar el texto de entrada. De nuevo, si el texto supera los 512 tokens, este se trunca dado que este es el límite de tokens que el modelo puede procesar.

In [ ]:
# Texto de ejemplo
text = threads_data['AskHistorians'][0]['description']

# Tokenizar el texto
input_ids = tokenizer(
    [WHITESPACE_HANDLER(text)],   # Preprocesar el texto
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

El siguiente paso consiste en generar el resumen del texto seleccionado, a partir de las `input_ids` obtenidos al tokenizar el texto.

In [ ]:
# Generar el resumen
output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,       # Longitud máxima del resumen generado
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

Por último, se debe decodificar el resumen, esto es, convertir las `output_ids` en texto.

In [ ]:
# Decodificar el resumen
summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

# Imprimir el resumen generado
print(summary)

Ahora que ya hemos visto cómo funciona el modelo, procedemos a aplicarlo a cada uno de los hilos seleccionados para generar sus respectivos resúmenes. Para ello, recorremos cada hilo en `threads_data` y procesamos su descripción para generar el resumen correspondiente.

In [ ]:
# Procesar los hilos
for subreddit, threads in threads_data.items():
    print(f"Procesando {len(threads)} hilos del subreddit {subreddit}...")

    for thread in threads:
        # Descripción del hilo
        text = thread['description']

        # Tokenizar el texto
        input_ids = tokenizer(
            [WHITESPACE_HANDLER(text)],   # Preprocesar el texto
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512
        )["input_ids"]

        # Generar el resumen
        output_ids = model.generate(
            input_ids=input_ids,
            max_length=84,       # Longitud máxima del resumen generado
            no_repeat_ngram_size=2,
            num_beams=4
        )[0]

        # Decodificar el resumen
        summary = tokenizer.decode(
            output_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )

        # Almacenar el resumen
        thread['summary'] = summary

print("Procesamiento de resúmenes completado.")

Mostramos ahora un ejemplo de un hilo, para ver que efectivamente se ha añadido el resumen, así como el sentimiento y la emoción del ejercicio 3.

In [ ]:
# Mostrar un ejemplo de hilo
print("Ejemplo de un hilo:", json.dumps(threads_data['AskHistorians'][0], indent=4, ensure_ascii=False))

Finalmente, modificamos los ficheros JSON añadiendo esta vez los resultados obtenidos tras haber almacenado los resúmenes de los diferentes hilos.

In [ ]:
# Obtener el directorio de trabajo actual
script_dir = os.getcwd()

# Ruta de la carpeta 'Modified Threads' dentro del directorio del script
modified_threads_dir = os.path.join(script_dir, 'Modified Threads')

# Crear la carpeta 'Modified Threads' si no existe
os.makedirs(modified_threads_dir, exist_ok=True)

# Iterar sobre los subreddits y sus hilos
for subreddit, threads in threads_data.items():
    # Crear una subcarpeta para cada subreddit
    subreddit_dir = os.path.join(modified_threads_dir, subreddit)
    os.makedirs(subreddit_dir, exist_ok=True)

    # Guardar los hilos modificados en la subcarpeta correspondiente
    for idx, thread in enumerate(threads, start=1):
        safe_name = limpiar_nombre_archivo(thread['title'])  # Limpiar el título para usarlo como nombre de archivo
        file_path = os.path.join(subreddit_dir, f'{safe_name}_modified_thread_{idx}.json')

        # Guardar el hilo modificado
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(thread, f, indent=4, ensure_ascii=False)

    print(f"Archivos del subreddit '{subreddit}' guardados en: {subreddit_dir}")

print(f"Todos los archivos han sido guardados en: {modified_threads_dir}")

### 4.2) Entrenando un nuevo modelo

En este segundo apartado, se pide hacer fine-tuning de modelos preentrenados. Para ello, probaremos con varios modelos, y finalmente haremos una comparación de resultados.

A continuación, cargamos el conjunto de datos que usaremos para hacer el fine-tuning de los modelos. En este caso, hemos escogido el dataset **Reddit TIFU**, y más concretamente su versión "short", un dataset que contiene publicaciones extraídas del subreddit r/tifu. Este se compone de historias narradas por los usuarios, con el título como un resumen del contenido de la publicación.

Hemos decidido usar este dataset debido a su relevancia para la tarea que debemos realizar, dado que este contiene ejemplos de resúmenes en un formato similar al que nuestros modelos deben generar. De hecho, estos datos están extraídos de Reddit, la misma red social con la que estamos trabajando, y puede ser útil para entrenar modelos en la generación de resúmenes en un contexto informal, similar al de nuestra tarea.

In [ ]:
# Cargar el conjunto de datos Reddit TIFU en su variante "short"
dataset = load_dataset("reddit_tifu", "short", trust_remote_code=True)

Debido a limitaciones computacionales, vamos a disminuir la longitud del dataset, tomando $3000$ ejemplos en total. Además, dividimos el dataset en un conjunto de entrenamiento, uno de validación y uno de test.

In [ ]:
# Seleccionar los primeros 3000 ejemplos del conjunto de entrenamiento
train_subset = dataset['train'].select(range(3000))

# Dividir el dataset en train (80%), val (10%) y test (10%)
train_dataset, temp_dataset = train_subset.train_test_split(test_size=0.2, shuffle=True, seed=42).values()
val_dataset, test_dataset = temp_dataset.train_test_split(test_size=0.5, shuffle=True, seed=42).values()

# Ver el tamaño de cada conjunto
print("Tamaño del conjunto de entrenamiento:", len(train_dataset))
print("Tamaño del conjunto de validación:", len(val_dataset))
print("Tamaño del conjunto de prueba:", len(test_dataset))

#### Modelo T5

Para comenzar, vamos a realizar un fine-tuning del modelo preentrenado **T5-small**, que es una versión más ligera del modelo T5, para la tarea de resumen automático.

En primer lugar, se especifica el modelo base, `t5-small`, y se cargan el tokenizador y el modelo preentrenado.

In [ ]:
# Definir el modelo
model_key = "t5-small"

# Cargar el tokenizador
tokenizer_t5 = T5Tokenizer.from_pretrained(model_key)

# Cargar el modelo pre-entrenado
model_t5 = T5ForConditionalGeneration.from_pretrained(model_key)

Una vez cargados los datos, definimos la función `preprocess_data_t5`, encargada de preprocesarlos. En primer lugar, esta función prepara cada texto del dataset añadiendo el prefijo `summarize: `, lo cual es necesario dado que el modelo T5 está diseñado para múltiples tareas y usa prefijos para diferenciarlas. Después, `tokenizer_t5` convierte los documentos y los respectivos resúmenes en tokens para que el modelo pueda procesarlos. Los resúmenes tokenizados se añaden como etiquetas indicando lo que el modelo debe predecir.

In [7]:
# Función de tokenización
def preprocess_data_t5(batch):
    # Preparar los textos de entrada con el prefijo "summarize: "
    inputs = ["summarize: " + text for text in batch["documents"]]

    # Tokenizar los textos de entrada
    model_inputs = tokenizer_t5(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenizar los resúmenes (títulos)
    labels = tokenizer_t5(batch["title"], max_length=128, truncation=True, padding="max_length")

    # Añadir los tokens del resumen como etiquetas
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Ahora, gracias a la función `preprocess_data_t5` que acabamos de definir, podemos tokenizar el dataset. Mediante `map` podemos aplicar dicha función a cada muestra del conjunto de entrenamiento y de validación.

In [ ]:
# Aplicar la tokenización
train_dataset_t5 = train_dataset.map(preprocess_data_t5, batched=True)
val_dataset_t5 = val_dataset.map(preprocess_data_t5, batched=True)

El siguiente paso consiste en definir los parámetros para el entrenamiento. Esto se puede hacer usando `TrainingArguments`, donde se definen, entre otros, los siguientes parámetros:

- `output_dir`: carpeta para guardar los resultados.

- `eval_strategy`: se evalúa el modelo al final de cada época.

- `learning_rate`: tasa de aprendizaje.

- `batch_size`: tamaño del lote para entrenamiento y para validación.

- `num_train_epochs`: número de épocas, esto es, de pasadas completas por el conjunto de entrenamiento.

- `weight_decay`: regularización para evitar sobreajuste.

In [ ]:
# Asegurarse de que existen las rutas necesarias
os.makedirs(os.path.join(os.getcwd(), 'results', 'results_T5'), exist_ok=True)
os.makedirs(os.path.join(os.getcwd(), 'logs', 'logs_T5'), exist_ok=True)

# Configurar argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir = os.path.join(os.getcwd(), 'results', 'results_T5'),
    eval_strategy = "epoch",
    learning_rate = 5e-5,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    num_train_epochs = 3,
    weight_decay = 0.01,
    save_total_limit = 2,
    logging_dir = os.path.join(os.getcwd(), 'logs', 'logs_T5'),
    logging_steps = 10,
    push_to_hub = False
)

A continuación, se configura la clase `Trainer`, la cual facilita el entrenamiento y la evaluación del modelo. Se le pasan el modelo, los argumentos de entrenamiento, los datasets y el tokenizador.

In [ ]:
# Configurar Trainer
trainer_t5 = Trainer(
    model = model_t5,
    args = training_args,
    train_dataset = train_dataset_t5,
    eval_dataset = val_dataset_t5,
    tokenizer = tokenizer_t5
)

Con todo esto, pasamos al entrenamiento y guardado del modelo.

In [ ]:
# Entrenar el modelo
trainer_t5.train()

# Guardar el modelo
os.makedirs(os.path.join(os.getcwd(), 'models', 'summarize_model_T5'), exist_ok=True)
trainer_t5.save_model(os.path.join(os.getcwd(), 'models', 'summarize_model_T5'))

Finalmente, para comprobar el funcionamiento del modelo, vamos a probarlo sobre la descripción del primer hilo de nuestra colección. En primer lugar, se debe tokenizar el texto de entrada, tomando una longitud máxima de 512 tokens, pues este es el límite de tokens que el modelo puede procesar. Mediante `model_t5.eval()`, se cambia el modelo a modo evaluación. Después, `generate` se encarga de generar un resumen a partir del texto tokenizado, y por último `decode` covierte los tokens generados a texto legible.

In [ ]:
# Texto de entrada
text = threads_data['AskHistorians'][0]['description']
inputs = tokenizer_t5("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

# Mover las entradas al dispositivo disponible
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generar resumen
model_t5.eval()
summary_ids = model_t5.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)
summary = tokenizer_t5.decode(summary_ids[0], skip_special_tokens=True)

print("Original:", text)
print("Resumen:", summary)

#### Modelo BART

El segundo modelo preentrenado al que vamos a hacerle un fine-tuning es el modelo **BART**, más específicamente su versión de tamaño base, debido a limitaciones computacionales. Este modelo es un transformer encoder-decoder preentrenado en inglés, que combina un encoder bidireccional similar a BERT, y un decoder autorregresivo como GPT. Su preentrenamiento consiste en dos pasos:

1. Corrupción del texto mediante una función de ruido arbitraria.
2. Aprendizaje de un modelo capaz de reconstruir el texto original.

Este preentrenamiento hace que BART sea particularmente adecuado para tareas de generación de texto, como resumen o traducción, además de tareas de comprensión de texto, como clasificación o respuesta a preguntas. Por esta misma razón hemos seleccionado BART, ya que es capaz de generar texto fluido y coherente, además de estar diseñado para tareas como el resumen automático, siendo ideal para adaptarlo mediante fine-tuning a nuestros datos.

Comenzamos especificando el modelo base, `facebook/bart-base`, y cargando el tokenizador y el modelo preentrenado.

In [ ]:
# Definir el modelo
model_key = "facebook/bart-base"

# Cargar el tokenizador
tokenizer_bart = BartTokenizer.from_pretrained(model_key)

# Cargar el modelo pre-entrenado
model_bart = BartForConditionalGeneration.from_pretrained(model_key)

Ahora, definimos la función `preprocess_data_bart`, encargada de preprocesar los datos cargados anteriormente. Esta función usa `tokenizer_bart` para convertir los documentos y los resúmenes en tokens que el modelo pueda procesar. Los resúmenes tokenizados se añaden como etiquetas indicando lo que el modelo debe predecir.

In [25]:
# Función de tokenización
def preprocess_data_bart(batch):

    # Tokenizar los textos de entrada
    model_inputs = tokenizer_bart(batch["documents"], max_length=512, truncation=True, padding="max_length")

    # Tokenizar los resúmenes (títulos)
    labels = tokenizer_bart(batch["title"], max_length=128, truncation=True, padding="max_length")

    # Añadir los tokens de los resúmenes como etiquetas
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Ahora, gracias a la función `preprocess_data_bart`, ya podemos tokenizar el dataset, usando `map` para aplicarla a cada muestra del conjunto de entrenamiento y de validación.

In [ ]:
# Aplicar la tokenización
train_dataset_bart = train_dataset.map(preprocess_data_bart, batched=True)
eval_dataset_bart = val_dataset.map(preprocess_data_bart, batched=True)

El siguiente paso consiste en definir los parámetros para el entrenamiento. En este caso, podemos reutilizar los argumentos definidos para el modelo anterior, usando `TrainingArguments`, que eran los siguientes:

In [ ]:
# Asegurarse de que existen las rutas necesarias
os.makedirs(os.path.join(os.getcwd(), 'results', 'results_BART'), exist_ok=True)
os.makedirs(os.path.join(os.getcwd(), 'logs', 'logs_BART'), exist_ok=True)

# Configurar argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir = os.path.join(os.getcwd(), 'results', 'results_BART'),
    eval_strategy = "epoch",
    learning_rate = 5e-5,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    num_train_epochs = 3,
    weight_decay = 0.01,
    save_total_limit = 2,
    logging_dir = os.path.join(os.getcwd(), 'logs', 'logs_BART'),
    logging_steps = 10,
    push_to_hub = False
)

A continuación, se configura la clase `Trainer` para facilitar el entrenamiento y la evaluación del modelo, pasándole el nuevo modelo, los argumentos de entrenamiento, los datasets y el tokenizador.

In [ ]:
# Configurar Trainer
trainer_bart = Trainer(
    model = model_bart,
    args = training_args,
    train_dataset = train_dataset_bart,
    eval_dataset = eval_dataset_bart,
    tokenizer = tokenizer_bart
)

Con todo esto, pasamos al entrenamiento y guardado del modelo.

In [ ]:
# Entrenar el modelo
trainer_bart.train()

# Guardar el modelo
os.makedirs(os.path.join(os.getcwd(), 'models', 'summarize_model_BART'), exist_ok=True)
trainer_bart.save_model(os.path.join(os.getcwd(), 'models', 'summarize_model_BART'))

Para finalizar, probamos el modelo sobre la descripción del primer hilo de nuestra colección. Para ello, se tokeniza el texto de entrada, tomando una longitud máxima de 512 tokens, y luego mediante `model_bart.eval()`, se cambia el modelo a modo evaluación. Después, `generate` se encarga de generar un resumen a partir del texto tokenizado, y por último `decode` covierte los tokens generados a texto legible.

In [ ]:
# Texto de entrada
text = threads_data['AskHistorians'][0]['description']
inputs = tokenizer_bart(text, return_tensors="pt", max_length=512, truncation=True)

# Mover las entradas al dispositivo disponible
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generar resumen
model_bart.eval()
summary_ids = model_bart.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)
summary = tokenizer_bart.decode(summary_ids[0], skip_special_tokens=True)

print("Original:", text)
print("Resumen:", summary)

#### Comparación de resultados

Una vez que hemos hecho fine-tuning de los dos modelos presentados, T5 y BART, ahora pasamos a comparar los resultados de dichos modelos utilizando la métrica ROUGE-N, que es una métrica muy adecuada para la evaluación de resúmenes automáticos. ROUGE-N compara la superposición de n-gramas (secuencias de n palabras consecutivas) entre los resúmenes generados por el modelo y los de referencia (en nuestro caso, los títulos de los hilos). Por ejemplo, con ROUGE-1 se comparan unigramas, esto es, palabras individuales, mientras que con ROUGE-2 se comparan bigramas, es decir, pares de palabras consecutivas. Además, ROUGE-L evalúa la longitud de la subsecuencia común más larga entre el resumen generado y el resumen de referencia.

Comenzamos cargando los modelos guardados anteriormente.

In [ ]:
# Cargar el modelo T5 y su tokenizador
model_t5 = T5ForConditionalGeneration.from_pretrained(os.path.join(os.getcwd(), 'models', 'summarize_model_T5'))
tokenizer_t5 = T5Tokenizer.from_pretrained(os.path.join(os.getcwd(), 'models', 'summarize_model_T5'))

# Cargar el modelo BART y su tokenizador
model_bart = BartForConditionalGeneration.from_pretrained(os.path.join(os.getcwd(), 'models', 'summarize_model_BART'))
tokenizer_bart = BartTokenizer.from_pretrained(os.path.join(os.getcwd(), 'models', 'summarize_model_BART'))

A continuación, se define una función que permite calcular la métrica ROUGE-N para evaluar la calidad de los resúmenes generados en comparación con los resúmenes de referencia. Utiliza el paquete `rouge_scorer` para obtener las métricas ROUGE-1, ROUGE-2 y ROUGE-L. Para ello, recorre las listas de predicciones y referencias, y para cada par, va calculando las métricas. Finalmente, se calcula el promedio de cada métrica y se devuelve como un diccionario.

In [15]:
# Función para calcular ROUGE-N
def compute_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = {"rouge1": [], "rouge2": [], "rougeL": []}

    for pred, ref in zip(predictions, references):
        score = scorer.score(ref, pred)
        for key in scores:
            scores[key].append(score[key].fmeasure)

    avg_scores = {key: sum(value) / len(value) for key, value in scores.items()}
    return avg_scores

Para empezar, vamos a evaluar el modelo con los datos de test del dataset Reddit TIFU, del que hemos extraido los datos de entrenamiento. Así, podremos obtener una idea del rendimiento del modelo en datos que siguen la misma distribución que los datos con los que ha sido entrenado. Comenzamos cargando dichos datos de test.

In [16]:
# Cargar datos de test de Reddit TIFU
test_texts = test_dataset["documents"]
test_references = test_dataset["title"]

Ahora, con estos datos, podemos generar los correspondientes resúmenes usando ambos modelos, T5 y BART.

In [22]:
# Mover el modelo al dispositivo disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_t5 = model_t5.to(device)

# Generar resúmenes con T5 para el dataset Reddit TIFU
predictions_t5 = []

for text in test_texts:
    # Tokenizar los textos
    inputs = tokenizer_t5("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Mover las entradas al dispositivo disponible
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generar el resumen
    summary_ids = model_t5.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)

    # Decodificar el resumen
    summary = tokenizer_t5.decode(summary_ids[0], skip_special_tokens=True)

    # Almacenar el resumen
    predictions_t5.append(summary)

In [30]:
# Mover el modelo al dispositivo disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_bart = model_bart.to(device)

# Generar resúmenes con BART para el dataset Reddit TIFU
predictions_bart = []

for text in test_texts:
    # Tokenizar los textos
    inputs = tokenizer_bart(text, return_tensors="pt", max_length=512, truncation=True)

    # Mover las entradas al dispositivo disponible
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generar el resumen
    summary_ids = model_bart.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)

    # Decodificar el resumen
    summary = tokenizer_bart.decode(summary_ids[0], skip_special_tokens=True)

    # Almacenar el resumen
    predictions_bart.append(summary)

Por último, ahora que ya tenemos los resúmenes, podemos evaluar el desempeño de los modelos usando las métricas ROUGE-N. Para ello, hacemos uso de la función `compute_rouge` definida anteriormente.

In [ ]:
# Evaluar ROUGE-N para T5
rouge_scores_t5 = compute_rouge(predictions_t5, test_references)

print("ROUGE-1 para T5:", rouge_scores_t5["rouge1"])
print("ROUGE-2 para T5:", rouge_scores_t5["rouge2"])
print("ROUGE-L para T5:", rouge_scores_t5["rougeL"])

In [ ]:
# Evaluar ROUGE-N para BART
rouge_scores_bart = compute_rouge(predictions_bart, test_references)

print("ROUGE-1 para BART:", rouge_scores_bart["rouge1"])
print("ROUGE-2 para BART:", rouge_scores_bart["rouge2"])
print("ROUGE-L para BART:", rouge_scores_bart["rougeL"])

Viendo los resultados obtenidos, en primer lugar podemos observar que los valores para el ROUGE-1, métrica que mide la superposición de palabras individuales entre las predicciones y las referencias, son ligeramente mejores para BART que para T5. Esto indica que BART captura más palabras clave relevantes en los resúmenes generados. Sin embargo, la diferencia no es significativa, lo que sugiere que ambos modelos están dando resultados razonablemente buenos.

Por otro lado, el ROUGE-2 evalúa la superposición de bigramas. Ambos modelos tienen puntuaciones similares, con T5 ligeramente superior. Esto indica que, a la hora de encontrar relaciones entre palabras consecutivas, T5 tiene una ligera ventaja. Sin embargo, ambas puntuaciones son bajas, lo que sugiere que ninguno de los modelos logra capturar completamente las relaciones entre pares de palabras consecutivas en el texto.

Por último, ROUGE-L mide como capta el modelo la estructura general y la fluidez del texto. En este caso, BART tiene una puntuación ligeramente superior, lo que indica que sus resúmenes siguen una estructura más cercana a las referencias en cuanto al orden y coherencia de las palabras.

En segundo lugar, vamos a comprobar el desempeño del modelo en nuestros hilos de Reddit, los cuales, si bien son similares a los del dataset Reddit TIFU usado para hacer el fine-tuning del modelo, puede que tengan algunas características diferentes. Así podremos evaluar si el modelo es capaz de adaptarse a nuestros datos específicos de Reddit, y ver cómo se comporta en diferentes contextos.

Comenzamos extrayendo las descripciones y los títulos de los hilos seleccionados.

In [40]:
# Extraer las descripciones y los títulos de los hilos
test_texts_reddit = []  # Lista para las descripciones
test_references_reddit = []  # Lista para los títulos

# Iterar sobre los subreddits y extraer los datos
for subreddit, threads in threads_data.items():
    for thread in threads:
        test_texts_reddit.append(thread['description'])
        test_references_reddit.append(thread['title'])

Una vez que tenemos las listas con las descripciones y con los títulos, podemos generar los resúmenes con T5 y con BART.

In [41]:
# Mover el modelo al dispositivo disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_t5 = model_t5.to(device)

# Generar resúmenes con T5 para nuestros datos de Reddit
predictions_t5_reddit = []

for text in test_texts_reddit:
    # Tokenizar los textos
    inputs = tokenizer_t5("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Mover las entradas al dispositivo disponible
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generar el resumen
    summary_ids = model_t5.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)

    # Decodificar el resumen
    summary = tokenizer_t5.decode(summary_ids[0], skip_special_tokens=True)

    # Almacenar el resumen
    predictions_t5_reddit.append(summary)

In [42]:
# Mover el modelo al dispositivo disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_bart = model_bart.to(device)

# Generar resúmenes con BART para nuestros datos de Reddit
predictions_bart_reddit = []

for text in test_texts_reddit:
    # Tokenizar los textos
    inputs = tokenizer_bart(text, return_tensors="pt", max_length=512, truncation=True)

    # Mover las entradas al dispositivo disponible
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generar el resumen
    summary_ids = model_bart.generate(inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)

    # Decodificar el resumen
    summary = tokenizer_bart.decode(summary_ids[0], skip_special_tokens=True)

    # Almacenar el resumen
    predictions_bart_reddit.append(summary)

Finalmente, con los resúmenes generados, podemos usar las métricas ROUGE-N para evaluar el desempeño de ambos modelos, utilizando la función `compute_rouge` definida anteriormente.

In [ ]:
# Evaluar ROUGE-N con T5 en nuestros datos de Reddit
rouge_scores_t5_reddit = compute_rouge(predictions_t5_reddit, test_references_reddit)

print("ROUGE-1 para T5 en Reddit:", rouge_scores_t5_reddit["rouge1"])
print("ROUGE-2 para T5 en Reddit:", rouge_scores_t5_reddit["rouge2"])
print("ROUGE-L para T5 en Reddit:", rouge_scores_t5_reddit["rougeL"])

In [ ]:
# Evaluar ROUGE-N con BART en nuestros datos de Reddit
rouge_scores_bart_reddit = compute_rouge(predictions_bart_reddit, test_references_reddit)

print("ROUGE-1 para BART en Reddit:", rouge_scores_bart_reddit["rouge1"])
print("ROUGE-2 para BART en Reddit:", rouge_scores_bart_reddit["rouge2"])
print("ROUGE-L para BART en Reddit:", rouge_scores_bart_reddit["rougeL"])

Al utilizar nuestros datos de Reddit, vemos que las métricas de ROUGE para T5 y BART disminuyen en comparación con el rendimiento con el dataset Reddit TIFU. Esto es esperable, e indica que el fine-tuning ha funcionado correctamente con los datos con que lo hemos llevado a cabo, extraídos del dataset Reddit TIFU, pero que al aplicar los modelos a nuestros datos personalizados, estos tienen dificultades para lograr resultados similares. Esto sugiere que las características del dataset personalizado podrían diferir de las del dataset Reddit TIFU (por ejemplo, longitud, complejidad o estilo).

Por ejemplo, para la métrica ROUGE-1, ambos modelos tienen puntuaciones más bajas en este conjunto personalizado que en Reddit TIFU. Sin embargo, T5 supera a BART, lo que indica que es ligeramente mejor al identificar términos relevantes en estos nuevos datos.

ROUGE-2 también muestra una caída considerable, especialmente para BART. Esto indica que los resúmenes generados por ambos modelos capturan muy pocas relaciones entre palabras consecutivas. La puntuación más alta de T5 sugiere que sigue siendo algo mejor en este sentido, pero ambos modelos tienen un rendimiento deficiente en esta métrica.

En el caso de ROUGE-L, ambas puntuaciones son bajas, aunque T5 supera ligeramente a BART, indicando que los resúmenes de T5 son algo más estructurados y se alinean mejor con el texto de referencia.

Por tanto, se observa que T5 generaliza algo mejor a nuestros datos personalizados, aunque su rendimiento es claramente superior con los datos de Reddit TIFU usados para el fine-tuning. Esto remarca la importancia del fine-tuning para adaptar un modelo preentrenado a un conjunto de datos específico.